In [4]:
import isx
from ci_pipe.pipeline import CIPipe
from ci_pipe.trace_builder import TraceBuilder

In [5]:
trace_builder = TraceBuilder(file_name="trace.json") # Replace with your desired trace file name
pipeline = CIPipe.with_videos_from_directory('input_dir', outputs_directory='output_dir', trace_builder=trace_builder, isx=isx) # Replace 'input_dir' and 'output_dir' with your actual directories

In [ ]:
(
    pipeline
    .isx.preprocess_videos()
    .isx.bandpass_filter_videos()
)

Bandpass filtering output_dir\Main Branch - Step 1 - ISX Preprocess Videos\simulation1-PP.isxd to output_dir\Main Branch - Step 2 - ISX Bandpass Filter Videos\simulation1-PP-BP.isxd, using low cutoff 0.007, high cutoff 0.6, retain mean False, subtract global minimum False
Bandpass filtering output_dir\Main Branch - Step 1 - ISX Preprocess Videos\simulation2-PP.isxd to output_dir\Main Branch - Step 2 - ISX Bandpass Filter Videos\simulation2-PP-BP.isxd, using low cutoff 0.007, high cutoff 0.6, retain mean False, subtract global minimum False
